In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
# 단어장(vocabulary) 만들기
vocab = list(set(sentence))
print(vocab)

['memory', 'Repeat', 'medicine', 'is', 'the', 'for', 'best']


In [4]:
# 단어장의 단어에 고유한 정수 인덱스 부여 // 모르는 단어 의미하는 UNK 토큰 추가
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0